In [29]:
import numpy as np
import cv2
import os
import time
from scipy.spatial import distance as distance
import cmath
import imutils
import base64
import requests

In [30]:
!pip3 install opencv-python

In [31]:
!pip3 install imutils

In [32]:
labelpath='darknet/data/coco.names'
file=open(labelpath)
label=file.read().strip().split("\n")
label[0]

'person'

In [33]:
weightspath=r'C:\Users\kunal\Desktop\Crowd-Detection-using-openCV\darknet\cfg\yolov3.weights'
configpath=r'C:\Users\kunal\Desktop\Crowd-Detection-using-openCV\darknet\cfg\yolov3.cfg'

In [34]:
net=cv2.dnn.readNetFromDarknet(configpath,weightspath)

In [35]:
videopath=r'C:\Users\kunal\Desktop\Crowd-Detection-using-openCV\video\run.mp4'

In [36]:
ln=net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]


In [37]:
URL_EDUCATIONAL = "http://things.ubidots.com"
URL_INDUSTRIAL = "http://industrial.api.ubidots.com"
INDUSTRIAL_USER = True  # Set this to False if you are an educational user
TOKEN = "BBFF-O8HEzPno59vPntQQJHq33AIhPk5Jlk"  # Put here your Ubidots TOKEN
DEVICE = "detector"  # Device where will be stored the result
VARIABLE = "people"  # Variable where will be stored the result



In [38]:
"""def buildPayload(variable, value):
    return {variable: {"value": value}}"""

'def buildPayload(variable, value):\n    return {variable: {"value": value}}'

In [39]:
def buildPayload(variable, value, context):
    return {variable: {"value": value, "context": context}}

In [40]:
def convert_to_base64(image):
    image = imutils.resize(image, width=300)
    img_str = cv2.imencode('.png', image)[1].tostring()
    b64 = base64.b64encode(img_str)

    return b64.decode('utf-8')

In [41]:
def sendToUbidots(token, device, variable, value, context={}, industrial=True):
    # Builds the endpoint
    url = URL_INDUSTRIAL if industrial else URL_EDUCATIONAL
    url = "{}/api/v1.6/devices/{}".format(url, device)

    payload = buildPayload(variable, value,context)
    headers = {"X-Auth-Token": token, "Content-Type": "application/json"}

    attempts = 0
    status = 400
   
    while status >= 400 and attempts <= 5:
        req = requests.post(url=url, headers=headers, json=payload)
        status = req.status_code
        attempts += 1
        time.sleep(1)
        print(req)

    return req

In [ ]:
video=cv2.VideoCapture(videopath)
ret = video
init=time.time()
sample_time=5
if sample_time < 1:
        sample_time = 1
while(True):
    
    
    ret,frame=video.read()
    if ret==False:
        print('Error running the file :(')
    frame=cv2.resize(frame, (640,440), interpolation =cv2.INTER_AREA)
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    r = blob[0, 0, :, :]
    net.setInput(blob)
    t0 = time.time()
    outputs = net.forward(ln)
    t = time.time()

    boxes = []
    confidences = []
    classIDs = []
    center=[]
    output=[]
    count=0
    results=[]
    breach=set()
    
    h, w = frame.shape[:2]
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
           
            confidence = scores[classID]
           
            if confidence > 0.5:
                box = detection[0:4] * np.array([w, h, w, h])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                center.append((centerX,centerY))
                box = [x, y, int(width), int(height)]
                boxes.append(box)
                confidences.append(float(confidence))
                classIDs.append(classID)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            #color = [int(c) for c in colors[classIDs[i]]]
            if(label[classIDs[i]]=='person'):
                #people()
                cX=(int)(x+(y/2))
                cY=(int)(w+(h/2))
                center.append((cX,cY))
                res=((x,y,x+w,y+h),center[i])
                results.append(res) 
                dist=cmath.sqrt(((center[i][0]-center[i+1][0])**2)+((center[i][1]-center[i+1][1])**2))
                if(dist.real <100):
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255), 2)
                    cv2.circle(frame,center[i],4,(0,0,255),-1)
                    #cv2.line(frame, (center[i][0], center[i][1]), (center[i+1][0], center[i+1][1]), (0,0, 255), thickness=3, lineType=8)
                    count=count+1
                    
                else:
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0), 2)
                    cv2.circle(frame,center[i],4,(0,255,0),-1)
        #cv2.rectangle(frame,(startX, startY), (endX, endY),color, 2)
        #cv2.circle(frame,(cX,cY),4,color,-1)
        cv2.putText(frame,"Violation: {}".format(count), (20, frame.shape[0] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (23,255,255), 1) 
        if time.time() - init >=sample_time:
            print("[INFO] Sending actual frame results")
            # Converts the image to base 64 and adds it to the context
            b64 = convert_to_base64(frame)
            context = {"image": b64}
            sendToUbidots(TOKEN, DEVICE,VARIABLE,count,context=context)
            init = time.time()
       

        
                    
    
    #cv2.putText(frame,"Violation: {}".format(count), (20, frame.shape[0] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (23,255,255), 1)      

    #frame_blob=makeblob(frame)
    cv2.imshow('Frame',frame)
    #cv2.VideoWriter(r'C:\Users\Lenovo\Desktop\Crowd Detection\video\walk2.mp4', cv2.VideoWriter_fourcc(*'MJPG'), 10, (640,440)).write(frame) 
    
    if(cv2.waitKey(1)==ord('q')):
        break
video.release()
cv2.destroyAllWindows()

[INFO] Sending actual frame results
<Response [200]>
[INFO] Sending actual frame results
<Response [200]>
[INFO] Sending actual frame results
<Response [200]>
[INFO] Sending actual frame results
<Response [200]>
[INFO] Sending actual frame results
<Response [200]>
[INFO] Sending actual frame results
<Response [200]>
[INFO] Sending actual frame results
<Response [200]>
[INFO] Sending actual frame results
<Response [200]>
[INFO] Sending actual frame results
<Response [200]>
[INFO] Sending actual frame results
<Response [200]>
[INFO] Sending actual frame results
<Response [200]>
[INFO] Sending actual frame results
<Response [200]>
